# Pipeline Complete

In [ ]:
import sys
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Always good to set a seed for reproducibility
SEED = 7
np.random.seed(SEED)

In [ ]:
# Loading Data
df = pd.read_csv('diabetes.csv')

# Getting dataframe columns names
df_name = df.columns
print(df_name)

In [ ]:
df.head()

In [ ]:
df.shape

## Analyze Data

### Descriptive Statitics

In [ ]:
df.info()

The data is not included time or object.

There is no null value in data set.

In [ ]:
df.describe()

In [ ]:
df['Outcome'].unique()

### Data Visualization

In [ ]:
g = sns.pairplot(df, hue="Outcome")

It seems that the data suffer from outliers

Let's see for example pregnancy distribution

In [ ]:
df["Pregnancies"].hist()

17 times pregnancy is a little bite strange, is it outlier ?

Let's investigate each feature distribution for each outcome

A fundamental task in many statistical analyses is to characterize the location and variability of a data set. A further characterization of the data includes skewness and kurtosis.

__Skewness__ is a measure of symmetry, or more precisely, the lack of symmetry. A distribution, or data set, is symmetric if it looks the same to the left and right of the center point.
- Negative values indicate data that are __skewed left__, i.e., the left tail is long relative to the right tail.
- Positive values indicate data that are __skewed right__, i.e., the right tail is long relative to the left tail.

__Kurtosis__ is a measure of whether the data are heavy-tailed or light-tailed relative to a normal distribution. That is, data sets with high kurtosis tend to have heavy tails, or outliers. Data sets with low kurtosis tend to have light tails, or lack of outliers. A uniform distribution would be the extreme case

In [ ]:
target = "Outcome"

for i in range(len(df_name[:-1])):
    plt.figure()
    df.groupby(target)[df_name[i]].plot(kind='hist',alpha=0.8,legend=True,title=df_name[i])

In [ ]:
from scipy.stats import skew
from scipy.stats import kurtosis

for i in range(len(df_name)):
    print('-*-'*25)
    print(df_name[i])
    print("{0} mean : ".format(df_name[i]), np.mean(df[df_name[i]]))
    print("{0} var  : ".format(df_name[i]), np.var(df[df_name[i]]))
    print("{0} skew : ".format(df_name[i]), skew(df[df_name[i]]))
    print("{0} kurt : ".format(df_name[i]), kurtosis(df[df_name[i]])) # >0 (curva più appuntita), <0 (curva più piatta)

In [ ]:
print(df.Outcome.value_counts())
sns.countplot(df, x='Outcome')
plt.show()

In the case of classification problem we always need to check the target distribution. If the distribution of target is not balance, we must treat our data more carefully. For example we can use several methods to resampling our data.In addition, we need to use stratified method in our validation in order to keep the same distribution in our train and test.

### Outliers investigation

In [ ]:
df.plot(kind='box', subplots=True, layout=(3,3))

In [ ]:
# for i in range(len(df_name[:-1])):
#     plt.figure()
#     df[df_name[i]].plot(kind='box')

### Evaluate Algorithms: Baseline
We usually devide the data to train and test set. We will not touch test set until the end of the computation and the final perpormance evaluation. Then, we can devide the train set to train and validation sets. We use the validation data set to tune the model.

Traditional train test method suffer from high variance test problem. It means that by changing the test set the result of the prediction changes. To overcome this problem we use k-fold validation method in our train and validation set

In [ ]:
from pandas import set_option
#from pandas.tools.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

Let's make train-validation and test data sets.

Note that stratify is used because we want to split the dataset by preserving the percentage of samples for each class.

In [ ]:
df_name[8]

In [ ]:
X =  df[df_name[0:8]]
Y = df[df_name[8]]
X_train, X_test, y_train, y_test =train_test_split(X,Y,
                                                   test_size=0.25,
                                                   random_state=0,
                                                   stratify=df['Outcome'])

In [ ]:
# Spot-Check Algorithms
def GetBasedModel():
    basedModels = []
    basedModels.append(('LR'   , LogisticRegression()))
    basedModels.append(('LDA'  , LinearDiscriminantAnalysis()))
    basedModels.append(('KNN'  , KNeighborsClassifier()))
    basedModels.append(('CART' , DecisionTreeClassifier()))
    basedModels.append(('NB'   , GaussianNB()))
    basedModels.append(('SVM'  , SVC(probability=True)))
    basedModels.append(('AB'   , AdaBoostClassifier()))
    basedModels.append(('GBM'  , GradientBoostingClassifier()))
    basedModels.append(('RF'   , RandomForestClassifier()))
    basedModels.append(('ET'   , ExtraTreesClassifier()))

    return basedModels

In [ ]:
def BasedLine2(X_train, y_train, models):
    # Test options and evaluation metric
    num_folds = 10
    scoring = 'accuracy'

    results = []
    names = []
    for name, model in models:
        kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=SEED)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)

        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)

    return names, results

In [ ]:
models = GetBasedModel()
names, results = BasedLine2(X_train, y_train, models)

In [ ]:
models

In [ ]:
names

In [ ]:
results

In [ ]:
comparison = pd.DataFrame(np.array(results).T, columns=names)
comparison

In [ ]:
comparison.plot(kind='box')

In [ ]:
def ScoreDataFrame(names, results):
    def floatingDecimals(f_val, dec=3):
        prc = "{:."+str(dec)+"f}"
        return float(prc.format(f_val))

    scores = []
    for r in results:
        scores.append(floatingDecimals(r.mean(), 4))

    scoreDataFrame = pd.DataFrame({'Model':names, 'Score': scores})
    return scoreDataFrame

In [ ]:
basedLineScore = ScoreDataFrame(names,results)
basedLineScore

### Feature Engineering
#### Data Preprocessig
Numerical features preprocessing is different for tree and non tree model.

1) Usually:

    - Tree based models does not depend on scaling
    - Non-tree based models hugely depend on scaling

2) Most Often used preprocessing are:

    - MinMax scaler to [0,1]
    - Standard Scaler to mean = 0 and std =1
    - Rank (We do not work on it in this data set)
    - Using np.log(1+data), np.sqrt(data) and stats.boxcox(data) (for exp dependency)

let's try some of them and see how our model prediction change by scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


def GetScaledModel(nameOfScaler):

    if nameOfScaler == 'standard':
        scaler = StandardScaler()
    elif nameOfScaler =='minmax':
        scaler = MinMaxScaler()

    pipelines = []
    pipelines.append((nameOfScaler+'LR'  , Pipeline([('Scaler', scaler),('LR'  , LogisticRegression())])))
    pipelines.append((nameOfScaler+'LDA' , Pipeline([('Scaler', scaler),('LDA' , LinearDiscriminantAnalysis())])))
    pipelines.append((nameOfScaler+'KNN' , Pipeline([('Scaler', scaler),('KNN' , KNeighborsClassifier())])))
    pipelines.append((nameOfScaler+'CART', Pipeline([('Scaler', scaler),('CART', DecisionTreeClassifier())])))
    pipelines.append((nameOfScaler+'NB'  , Pipeline([('Scaler', scaler),('NB'  , GaussianNB())])))
    pipelines.append((nameOfScaler+'SVM' , Pipeline([('Scaler', scaler),('SVM' , SVC())])))
    pipelines.append((nameOfScaler+'AB'  , Pipeline([('Scaler', scaler),('AB'  , AdaBoostClassifier())])  ))
    pipelines.append((nameOfScaler+'GBM' , Pipeline([('Scaler', scaler),('GMB' , GradientBoostingClassifier())])  ))
    pipelines.append((nameOfScaler+'RF'  , Pipeline([('Scaler', scaler),('RF'  , RandomForestClassifier())])  ))
    pipelines.append((nameOfScaler+'ET'  , Pipeline([('Scaler', scaler),('ET'  , ExtraTreesClassifier())])  ))


    return pipelines

### Standard

In [ ]:
models = GetScaledModel('standard')
names,results = BasedLine2(X_train, y_train, models)
comparison = pd.DataFrame(np.array(results).T, columns=names)

print(comparison.head())
comparison.plot(kind='box')

scaledScoreStandard = ScoreDataFrame(names,results)
compareModels = pd.concat([basedLineScore,
                           scaledScoreStandard], axis=1)
compareModels

### MinMax

In [ ]:
models = GetScaledModel('minmax')
names,results = BasedLine2(X_train, y_train,models)
comparison = pd.DataFrame(np.array(results).T, columns=names)

print(comparison.head())
comparison.plot(kind='box')

scaledScoreMinMax = ScoreDataFrame(names,results)
compareModels = pd.concat([basedLineScore,
                           scaledScoreStandard,
                          scaledScoreMinMax], axis=1)
compareModels

As we can see, the standarscaler and min and max effect on non tree models and the prediction results improve

### Remove Outliers
let's remove outliers and see how it effects on the prediction

In [ ]:
df_t = df.copy()
df_t_name = df_t.columns

In [ ]:
def TurkyOutliers(df_out,nameOfFeature,drop=False):

    valueOfFeature = df_out[nameOfFeature]
    # Calculate Q1 (25th percentile of the data) for the given feature
    Q1 = np.percentile(valueOfFeature, 25.)

    # Calculate Q3 (75th percentile of the data) for the given feature
    Q3 = np.percentile(valueOfFeature, 75.)

    # Use the interquartile range to calculate an outlier step (1.5 times the interquartile range)
    step = (Q3-Q1)*1.5
    # print "Outlier step:", step
    outliers = valueOfFeature[~((valueOfFeature >= Q1 - step) & (valueOfFeature <= Q3 + step))].index.tolist()
    feature_outliers = valueOfFeature[~((valueOfFeature >= Q1 - step) & (valueOfFeature <= Q3 + step))].values
    # df[~((df[nameOfFeature] >= Q1 - step) & (df[nameOfFeature] <= Q3 + step))]


    # Remove the outliers, if any were specified
    print ("Number of outliers (inc duplicates): {} and outliers: {}".format(len(outliers), feature_outliers))
    if drop:
        good_data = df_out.drop(df_out.index[outliers]).reset_index(drop = True)
        print ("New dataset with removed outliers has {} samples with {} features each.".format(*good_data.shape))
        return good_data
    else:
        print ("Nothing happens, df.shape = ",df_out.shape)
        return df_out

In [ ]:
df_t_name

In [ ]:
#feature_number = 0
#OutLiersBox(df,df_name[feature_number])

for i in range(len(df_name[:-1])):
    plt.figure()
    df[df_name[i]].plot(kind='box')

    df_clean = TurkyOutliers(df, df_name[i], True)

    plt.figure()
    df_clean[df_name[i]].plot(kind='box')

In [ ]:
df_clean = df.copy()
for i in range(len(df_name[:-1])):
    print(df_name[i])
    df_clean = TurkyOutliers(df_clean,df_name[i],True)

In [ ]:
print('df shape: {}, new df shape: {}, we lost {} rows, {}% of our data'.format(df.shape[0],df_clean.shape[0],
                                                              df.shape[0]-df_clean.shape[0],
                                                        (df.shape[0]-df_clean.shape[0])/df.shape[0]*100))

### Comparing the accuracy of models after cleaning

In [ ]:
df_clean_name = df_clean.columns
X_c =  df_clean[df_clean_name[0:8]]
Y_c = df_clean[df_clean_name[8]]

X_train_c, X_test_c, y_train_c, y_test_c =train_test_split(X_c,Y_c,
                                                   test_size=0.25,
                                                   random_state=0,
                                                   stratify=df_clean['Outcome'])

In [ ]:
models = GetScaledModel('minmax')
names,results = BasedLine2(X_train_c, y_train_c,models)
comparison = pd.DataFrame(np.array(results).T, columns=names)
print(comparison.head())
comparison.plot(kind='box')

scaledScoreMinMax_c = ScoreDataFrame(names,results)
compareModels = pd.concat([basedLineScore,
                           scaledScoreStandard,
                          scaledScoreMinMax,
                          scaledScoreMinMax_c], axis=1)
compareModels

It can be seen that the prediction is improving.

### Feature Selection
Feature selection is also called variable selection or attribute selection. It is the automatic selection of attributes in your data (such as columns in tabular data) that are most relevant to the predictive modeling problem you are working on.

Feature selection methods aid you in your mission to create an accurate predictive model. They help you by choosing features that will give you as good or better accuracy whilst requiring less data.

Feature selection methods can be used to identify and remove unneeded, irrelevant and redundant attributes from data that do not contribute to the accuracy of a predictive model or may in fact decrease the accuracy of the model.

### Correlation
If we fit highly corrolated data in our model, it results in the overfitting probelm. Thus, for example if there are two highly corrolated features we have to drop the one that has more corrolation with other feature.

In [ ]:
sns.heatmap(df.corr(), annot=True)

There is not highly corrolated feature in this data set.

### Feature Importance
Bagged decision trees like Random Forest and Extra Trees can be used to estimate the importance of features.

In [ ]:
clf = ExtraTreesClassifier(n_estimators=250,
                              random_state=SEED)

clf.fit(X_train_c, y_train_c)

# #############################################################################
# Plot feature importance
feature_importance = clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)

pos = np.arange(sorted_idx.shape[0]) + .5
# plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, df.columns[sorted_idx])#boston.feature_names[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

In [ ]:
df_feature_imp = df_clean[['Glucose','BMI','Age','DiabetesPedigreeFunction','Outcome']]
df_feature_imp_name = df_feature_imp.columns

In [ ]:
X =  df_feature_imp[df_feature_imp_name[0 : df_feature_imp.shape[1] - 1]]
Y = df_feature_imp[df_feature_imp_name[df_feature_imp.shape[1] - 1]]

X_train_im, X_test_im, y_train_im, y_test_im =train_test_split(X,Y,
                                                   test_size=0.1,
                                                   random_state=0,
                                                   stratify=df_feature_imp['Outcome'])

In [ ]:
models = GetScaledModel('minmax')
names,results = BasedLine2(X_train_im, y_train_im,models)
comparison = pd.DataFrame(np.array(results).T, columns=names)
print(comparison.head())
comparison.plot(kind='box')

scaledScoreMinMax_im = ScoreDataFrame(names,results)
compareModels = pd.concat([basedLineScore,
                           scaledScoreStandard,
                          scaledScoreMinMax,
                          scaledScoreMinMax_c,
                          scaledScoreMinMax_im], axis=1)
compareModels

We still could improve the prediction

### Algortithm Tuning

In [ ]:
df_unscaled = df_clean[['Glucose', 'BMI', 'Age', 'DiabetesPedigreeFunction', 'Outcome']]
df_imp_scaled_name = df_unscaled.columns

In [ ]:
df_imp_scaled = MinMaxScaler().fit_transform(df_unscaled)
X =  df_imp_scaled[:, 0:4]
Y =  df_imp_scaled[:, 4]
X_train_sc, X_test_sc, y_train_sc, y_test_sc = train_test_split(X, Y,
                                                                test_size=0.1,
                                                                random_state=0,
                                                                stratify=df_imp_scaled[:, 4])

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform

In [ ]:
class RandomSearch(object):

    def __init__(self ,X_train, y_train, model, hyperparameters):

        self.X_train = X_train
        self.y_train = y_train
        self.model = model
        self.hyperparameters = hyperparameters

    def RandomSearch(self):
        # Create randomized search 10-fold cross validation and 100 iterations
        cv = 10
        clf = RandomizedSearchCV(self.model,
                                 self.hyperparameters,
                                 random_state=1,
                                 n_iter=100,
                                 cv=cv,
                                 verbose=0,
                                 n_jobs=-1,
                                 )
        # Fit randomized search
        best_model = clf.fit(self.X_train, self.y_train)
        message = (best_model.best_score_, best_model.best_params_)
        print("Best: %f using %s" % (message))

        return best_model, best_model.best_params_

    def BestModelPridict(self, X_test):

        best_model, _ = self.RandomSearch()
        pred = best_model.predict(X_test)
        return pred

In [ ]:
class GridSearch(object):

    def __init__(self, X_train, y_train, model, hyperparameters):

        self.X_train = X_train
        self.y_train = y_train
        self.model = model
        self.hyperparameters = hyperparameters

    def GridSearch(self):
        # Create randomized search 10-fold cross validation
        cv = 10
        clf = GridSearchCV(self.model,
                           self.hyperparameters,
                           cv=cv,
                           verbose=0,
                           n_jobs=-1,
                           )

        # Fit grid search
        best_model = clf.fit(self.X_train, self.y_train)
        message = (best_model.best_score_, best_model.best_params_)
        print("Best: %f using %s" % (message))

        return best_model, best_model.best_params_

    def BestModelPridict(self,X_test):

        best_model, _ = self.GridSearch()
        pred = best_model.predict(X_test)
        return pred

#### Logistic Regression
    - C : Regularization value, the more, the stronger the regularization(double).
    - RegularizationType: Can be either "L2" or “L1”. Default is “L2”.

In [ ]:
uniform(loc=0, scale=4)

In [ ]:
# model
model = LogisticRegression()
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter distribution using uniform distribution
C = uniform(loc=0, scale=4)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

In [ ]:
hyperparameters

In [ ]:
LR_RandSearch = RandomSearch(X_train_sc, y_train_sc, model, hyperparameters)
# LR_best_model, LR_best_params = LR_RandSearch.RandomSearch()

Prediction_LR = LR_RandSearch.BestModelPridict(X_test_sc)

In [ ]:
def floatingDecimals(f_val, dec=3):
  prc = "{:."+str(dec)+"f}" #first cast decimal as str
  # print(prc) #str format output is {:.3f}
  return float(prc.format(f_val))

print('prediction on test set is:', floatingDecimals((y_test_sc==Prediction_LR).mean(), 7))

### KNN
    - n_neighbors: Number of neighbors to use by default for k_neighbors queries

In [ ]:
model_KNN = KNeighborsClassifier()

neighbors = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
param_grid = dict(n_neighbors=neighbors)

In [ ]:
KNN_GridSearch = GridSearch(X_train_sc, y_train_sc, model_KNN, param_grid)
Prediction_KNN = KNN_GridSearch.BestModelPridict(X_test_sc)
print('prediction on test set is:' ,floatingDecimals((y_test_sc == Prediction_KNN).mean(), 7))

### SVC
    - C: The Penalty parameter C of the error term.
    - Kernel: Kernel type could be linear, poly, rbf or sigmoid.

In [ ]:
c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
kernel_values = [ 'linear' , 'poly' , 'rbf' , 'sigmoid' ]
param_grid = dict(C=c_values, kernel=kernel_values)
model_SVC = SVC()

In [ ]:
SVC_GridSearch = GridSearch(X_train_sc,y_train_sc,model_SVC,param_grid)
Prediction_SVC = SVC_GridSearch.BestModelPridict(X_test_sc)
print('prediction on test set is:', floatingDecimals((y_test_sc == Prediction_SVC).mean(), 7))

### Decision Tree
    - max_depth: Maximum depth of the tree (double).
    - row_subsample: Proportion of observations to consider (double).
    - max_features: Proportion of columns (features) to consider in each level (double).

In [ ]:
from scipy.stats import randint
max_depth_value = [3, None]
max_features_value =  randint(1, 4)
min_samples_leaf_value = randint(1, 4)
criterion_value = ["gini", "entropy"]

In [ ]:
param_grid = dict(max_depth = max_depth_value,
                  max_features = max_features_value,
                  min_samples_leaf = min_samples_leaf_value,
                  criterion = criterion_value)

In [ ]:
model_CART = DecisionTreeClassifier()
CART_RandSearch = RandomSearch(X_train_sc,y_train_sc,model_CART,param_grid)
Prediction_CART = CART_RandSearch.BestModelPridict(X_test_sc)
print('prediction on test set is:' ,floatingDecimals((y_test_sc == Prediction_CART).mean(),7))

### AdaBoostClassifier
    - learning_rate: Learning rate shrinks the contribution of each classifier by learning_rate.
    - n_estimators: Number of trees to build.

In [ ]:
learning_rate_value = [.01,.05,.1,.5,1]
n_estimators_value = [50,100,150,200,250,300]

param_grid = dict(learning_rate=learning_rate_value, n_estimators=n_estimators_value)

In [ ]:
model_Ad = AdaBoostClassifier()
Ad_GridSearch = GridSearch(X_train_sc,y_train_sc,model_Ad,param_grid)
Prediction_Ad = Ad_GridSearch.BestModelPridict(X_test_sc)
print('prediction on test set is:' ,floatingDecimals((y_test_sc == Prediction_Ad).mean(),7))

### GradientBoosting

In [ ]:
learning_rate_value = [.01,.05,.1,.5,1]
n_estimators_value = [50,100,150,200,250,300]

param_grid = dict(learning_rate=learning_rate_value, n_estimators=n_estimators_value)

In [ ]:
model_GB = GradientBoostingClassifier()
GB_GridSearch = GridSearch(X_train_sc,y_train_sc,model_GB,param_grid)
Prediction_GB = GB_GridSearch.BestModelPridict(X_test_sc)
print('prediction on test set is:' ,floatingDecimals((y_test_sc == Prediction_GB).mean(),7))

### Ensemble Methods

### Voting Ensemble
Voting is one of the simplest ways of combining the predictions from multiple machine learning algorithms. It works by first creating two or more standalone models from your training dataset. A Voting Classifier can then be used to wrap your models and average the predictions of the sub-models when asked to make predictions for new data.

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
param = {'C': 0.7678243129497218, 'penalty': 'l2'}
model1 = LogisticRegression(**param)

param = {'n_neighbors': 15}
model2 = KNeighborsClassifier(**param)

param = {'C': 1.7, 'kernel': 'linear'}
model3 = SVC(**param)

param = {'criterion': 'gini', 'max_depth': 3, 'max_features': 2, 'min_samples_leaf': 3}
model4 = DecisionTreeClassifier(**param)

param = {'learning_rate': 0.05, 'n_estimators': 150}
model5 = AdaBoostClassifier(**param)

param = {'learning_rate': 0.01, 'n_estimators': 100}
model6 = GradientBoostingClassifier(**param)

model7 = GaussianNB()

model8 = RandomForestClassifier()

model9 = ExtraTreesClassifier()

In [ ]:
# create the sub models
estimators = [('LR',model1), ('KNN',model2), ('SVC',model3),
              ('DT',model4), ('ADa',model5), ('GB',model6),
              ('NB',model7), ('RF',model8),  ('ET',model9)]

In [ ]:
# create the ensemble model
kfold = StratifiedKFold(n_splits=10, random_state=SEED, shuffle=True)
ensemble = VotingClassifier(estimators)
results = cross_val_score(ensemble, X_train_sc, y_train_sc, cv=kfold)

print('Accuracy on train: ', results.mean())

ensemble_model = ensemble.fit(X_train_sc, y_train_sc)
pred = ensemble.predict(X_test_sc)

print('Accuracy on test:' , (y_test_sc == pred).mean())